In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt 
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error

TICKER_DIR = "C:\\Users\\anton\\Documents\\antoniouaa\\msc_thesis\\data\\tickers\\ticker_data\\_Rolling"
# datasets = []
# for f in os.listdir(TICKER_DIR):
#     df = pd.read_csv(os.path.join(TICKER_DIR, f), header=0)
#     datasets.append(df)

# dataset = pd.concat(datasets)
# dataset = dataset.dropna(how="any")
dataset = pd.read_csv(os.path.join(TICKER_DIR, "MA_AIZP.csv"))
dataset = dataset.dropna(how="any")
dataset

,close,high,low,open,volume,R.Mean,R.std,S0%k,EMA,BolBand1,BolBand2
13,102.39,103.28,102.22,102.641,4451,102.100121,0.917114,63.636364,102.249329,0.035930,0.579019
14,103.54,105.00,103.14,103.140,5098,102.310121,0.883166,100.000000,102.444197,0.034529,0.848145
15,104.15,104.23,103.25,103.250,4766,102.472979,0.998456,100.000000,102.697275,0.038974,0.919904
16,103.00,104.40,101.28,103.460,15545,102.547979,0.995582,67.877095,102.741524,0.038834,0.613507
17,103.19,103.58,101.87,103.580,15521,102.682264,0.941052,73.184358,102.806245,0.036659,0.634885
...,...,...,...,...,...,...,...,...,...,...,...
182,107.87,107.87,106.67,106.670,1377,108.317857,1.872786,37.064220,107.658271,0.069159,0.440215
183,106.61,107.12,106.61,107.120,14200,108.332857,1.857018,13.944954,107.518502,0.068567,0.268061
184,106.66,106.84,104.03,105.850,18721,108.095000,1.841633,14.862385,107.404035,0.068149,0.305200
185,105.00,105.00,105.00,105.000,25240,107.738571,1.927541,0.000000,107.083497,0.071564,0.144810


In [2]:
def evaluate_arima(X, order):
    size = int(len(X) * 0.66)
    train, test = X[:size], X[size:]
    history = [x for x in train]
    predictions = []
    for t in range(len(test)):
        model = ARIMA(history, order=order)
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        history.append(test[t])
        # print(f"predicted={yhat}, expected={t}")

    error = mean_squared_error(test, predictions)
    return error

In [44]:
def evaluate_models(X, p_values, d_values, q_values):
    scores = []
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p, d, q)
                try:
                    mse = evaluate_arima(X, order)
                    scores.append((mse, order))
                except:
                    continue
    return scores
    
X = dataset["close"].values
scores = evaluate_models(X, [0, 1, 2, 4, 6, 8, 10], range(3), range(3))
best_score = sorted(scores)
print(best_score)
print(f"Best ARIMA({best_score[0]}), MSE={best_score[1]}")

ARIMA((0, 0, 0)) MSE=9.160582980958376
ARIMA((0, 0, 1)) MSE=4.259077347239624
ARIMA((0, 0, 2)) MSE=2.4418956800196985
ARIMA((0, 1, 0)) MSE=1.8158156174911142
c:\Users\anton\Documents\antoniouaa\msc_thesis\venv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\Users\anton\Documents\antoniouaa\msc_thesis\venv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\Users\anton\Documents\antoniouaa\msc_thesis\venv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
c:\Users\anton\Documents\antoniouaa\msc_thesis\venv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood 